# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [1]:
#Import your libraries

import numpy as np
import pandas as pd

# Introduction

In this lab, we will use two datasets. Both datasets contain variables that describe apps from the Google Play Store. We will use our knowledge in feature extraction to process these datasets and prepare them for the use of a ML algorithm.

# Challenge 1 - Loading and Extracting Features from the First Dataset

#### In this challenge, our goals are: 

* Exploring the dataset.
* Identify the columns with missing values.
* Either replacing the missing values in each column or drop the columns.
* Conver each column to the appropriate type.

#### The first dataset contains different information describing the apps. 

Load the dataset into the variable `google_play` in the cell below. The dataset is in the file `googleplaystore.csv`

In [2]:
# Your code here:

google_play = pd.read_csv('../googleplaystore.csv')

#### Examine all variables and their types in the following cell

In [3]:
# Your code here:
google_play.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

#### Since this dataset only contains one numeric column, let's skip the `describe()` function and look at the first 5 rows using the `head()` function

In [4]:
# Your code here:
google_play.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


#### We can see that there are a few columns that could be coerced to numeric.

Start with the reviews column. We can evaluate what value is causing this column to be of object type finding the non-numeric values in this column. To do this, we recall the `to_numeric()` function. With this function, we are able to coerce all non-numeric data to null. We can then use the `isnull()` function to subset our dataframe using the True/False column that this function generates.

In the cell below, transform the Reviews column to numeric and assign this new column to the variable `Reviews_numeric`. Make sure to coerce the errors.

In [5]:
# Your code here:
google_play['Reviews_numeric']=google_play.Reviews.apply(pd.to_numeric, errors='coerce')

In [6]:
google_play['Reviews_numeric'].dtypes

dtype('float64')

In [7]:
google_play['Reviews_numeric'].head()

0       159.0
1       967.0
2     87510.0
3    215644.0
4       967.0
Name: Reviews_numeric, dtype: float64

Next, create a column containing True/False values using the `isnull()` function. Assign this column to the `Reviews_isnull` variable.

In [8]:
# Your code here:
google_play['Reviews_isnull'] = google_play.Reviews_numeric.isnull()

In [9]:
google_play.Reviews_isnull.head()

0    False
1    False
2    False
3    False
4    False
Name: Reviews_isnull, dtype: bool

Finally, subset the `google_play` with `Reviews_isnull`. This should give you all the rows that contain non-numeric characters.

Your output should look like:

![Reviews_bool.png](reviews-bool.png)

In [10]:
# Your code here:
google_play[google_play['Reviews_isnull']]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Reviews_numeric,Reviews_isnull
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN,NaN,True


#### We see that Google Play is using a shorthand for millions. 

Let's write a function to transform this data.

Steps:

1. Create a function that returns the correct numeric values of *Reviews*.
1. Define a test string with `M` in the last character.
1. Test your function with the test string. Make sure your function works correctly. If not, modify your functions and test again.

In [11]:
# Your code here

def convert_string_to_numeric(s):
    """
    Convert a string value to numeric. If the last character of the string is `M`, obtain the 
    numeric part of the string, multiply it with 1,000,000, then return the result. Otherwise, 
    convert the string to numeric value and return the result.
    
    Args:
        s: The Reviews score in string format.
    
    Returns:
        The correct numeric value of the Reviews score.
    """
    if (type(s)==str) and (s[-1] == 'M'):
        s=float(s[:-1])*1000000
    else :
        s=float(s)
    return s

test_string = '4.0M'

convert_string_to_numeric(test_string) == 4000000

True

The last step is to apply the function to the `Reviews` column in the following cell:

In [12]:
# Your code here:
google_play.Reviews=google_play.Reviews.apply(convert_string_to_numeric)

Check the non-numeric `Reviews` row again. It should have been fixed now and you should see:

![Reviews_bool_fixed.png](reviews-bool-fixed.png)

In [13]:
# Your code here
google_play.iloc[[10472,]].Reviews

10472    3000000.0
Name: Reviews, dtype: float64

Also check the variable types of `google_play`. The `Reviews` column should be a `float64` type now.

In [14]:
# Your code here:
google_play.dtypes

App                 object
Category            object
Rating             float64
Reviews            float64
Size                object
Installs            object
Type                object
Price               object
Content Rating      object
Genres              object
Last Updated        object
Current Ver         object
Android Ver         object
Reviews_numeric    float64
Reviews_isnull        bool
dtype: object

#### The next column we will look at is `Size`. We start by looking at all unique values in `Size`:

*Hint: use `unique()` ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html))*.

In [15]:
# Your code here:
google_play.Size.unique()

array(['19M', '14M', '8.7M', '25M', '2.8M', '5.6M', '29M', '33M', '3.1M',
       '28M', '12M', '20M', '21M', '37M', '2.7M', '5.5M', '17M', '39M',
       '31M', '4.2M', '7.0M', '23M', '6.0M', '6.1M', '4.6M', '9.2M',
       '5.2M', '11M', '24M', 'Varies with device', '9.4M', '15M', '10M',
       '1.2M', '26M', '8.0M', '7.9M', '56M', '57M', '35M', '54M', '201k',
       '3.6M', '5.7M', '8.6M', '2.4M', '27M', '2.5M', '16M', '3.4M',
       '8.9M', '3.9M', '2.9M', '38M', '32M', '5.4M', '18M', '1.1M',
       '2.2M', '4.5M', '9.8M', '52M', '9.0M', '6.7M', '30M', '2.6M',
       '7.1M', '3.7M', '22M', '7.4M', '6.4M', '3.2M', '8.2M', '9.9M',
       '4.9M', '9.5M', '5.0M', '5.9M', '13M', '73M', '6.8M', '3.5M',
       '4.0M', '2.3M', '7.2M', '2.1M', '42M', '7.3M', '9.1M', '55M',
       '23k', '6.5M', '1.5M', '7.5M', '51M', '41M', '48M', '8.5M', '46M',
       '8.3M', '4.3M', '4.7M', '3.3M', '40M', '7.8M', '8.8M', '6.6M',
       '5.1M', '61M', '66M', '79k', '8.4M', '118k', '44M', '695k', '1.6M',
     

You should have seen lots of unique values of the app sizes.

#### While we can convert most of the `Size` values to numeric in the same way we converted the `Reviews` values, there is one value that is impossible to convert.

What is that badass value? Enter it in the next cell and calculate the proportion of its occurence to the total number of records of `google_play`.

In [16]:
# Your code here:
(google_play[google_play['Size']=='Varies with device'].shape[0]/google_play.shape[0])*100

15.635089013928605

#### While this column may be useful for other types of analysis, we opt to drop it from our dataset. 

There are two reasons. First, the majority of the data are ordinal but a sizeable proportion are missing because we cannot convert them to numerical values. Ordinal data are both numerical and categorical, and they usually can be ranked (e.g. 82k is smaller than 91M). In contrast, non-ordinal categorical data such as blood type and eye color cannot be ranked. The second reason is as a categorical column, it has too many unique values to produce meaningful insights. Therefore, in our case the simplest strategy would be to drop the column.

Drop the column in the cell below (use `inplace=True`)

In [17]:
# Your code here:
google_play.drop(['Size'], inplace=True, axis=1)

In [18]:
google_play.shape

(10841, 14)

#### Now let's look at how many missing values are in each column. 

This will give us an idea of whether we should come up with a missing data strategy or give up on the column all together. In the next column, find the number of missing values in each column: 

*Hint: use the `isna()` and `sum()` functions.*

In [19]:
# Your code here:
google_play.isna().sum()

App                   0
Category              0
Rating             1474
Reviews               0
Installs              0
Type                  1
Price                 0
Content Rating        1
Genres                0
Last Updated          0
Current Ver           8
Android Ver           3
Reviews_numeric       1
Reviews_isnull        0
dtype: int64

You should find the column with the most missing values is now `Rating`.

#### What is the proportion of the missing values in `Rating` to the total number of records?

Enter your answer in the cell below.

In [20]:
# Your code here:
round(google_play.Rating.isna().sum()/google_play.shape[0],2)*100

14.000000000000002

A sizeable proportion of the `Rating` column is missing. A few other columns also contain several missing values.

#### We opt to preserve these columns and remove the rows containing missing data.

In particular, we don't want to drop the `Rating` column because:

* It is one of the most important columns in our dataset. 

* Since the dataset is not a time series, the loss of these rows will not have a negative impact on our ability to analyze the data. It will, however, cause us to lose some meaningful observations. But the loss is limited compared to the gain we receive by preserving these columns.

In the cell below, remove all rows containing at least one missing value. Use the `dropna()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)). Assign the new dataframe to the variable `google_missing_removed`.

In [49]:
# Your code here:
google_missing_removed = google_play.dropna(subset=['Rating'],axis=0)

In [50]:
google_missing_removed.isna().sum()

App                0
Category           0
Rating             0
Reviews            0
Installs           0
Type               0
Price              0
Content Rating     1
Genres             0
Last Updated       0
Current Ver        4
Android Ver        3
Reviews_numeric    1
Reviews_isnull     0
dtype: int64

From now on, we use the `google_missing_removed` variable instead of `google_play`.

#### Next, we look at the `Last Updated` column.

The `Last Updated` column seems to contain a date, though it is classified as an object type. Let's convert this column using the `pd.to_datetime` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)).

In [51]:
google_missing_removed['Last Updated']

0           January 7, 2018
1          January 15, 2018
2            August 1, 2018
3              June 8, 2018
4             June 20, 2018
5            March 26, 2017
6            April 26, 2018
7             June 14, 2018
8        September 20, 2017
9              July 3, 2018
10         October 27, 2017
11            July 31, 2018
12            April 2, 2018
13            June 26, 2018
14           August 3, 2018
15             June 6, 2018
16            July 31, 2018
17         November 7, 2017
18           August 3, 2018
19            July 30, 2018
20           April 20, 2018
21           March 20, 2018
22            July 12, 2018
24             July 7, 2018
25           April 25, 2018
26         October 11, 2017
27           March 21, 2018
28            July 12, 2018
29          August 22, 2017
30            July 12, 2018
                ...        
10795          May 10, 2018
10796         July 26, 2018
10797         June 26, 2018
10799          May 20, 2018
10800         July 3

In [52]:
google_missing_removed['Last Updated'] = pd.to_datetime(google_missing_removed['Last Updated'], format='%B %d, %Y', errors='coerce')

In [53]:
# Your code here:
google_missing_removed = google_missing_removed.dropna(subset=['Last Updated'],axis=0)

#### The last column we will transform is `Price`. 

We start by looking at the unique values of this column.

In [54]:
# Your code here:
google_missing_removed['Price'].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$7.99', '$5.99', '$2.99', '$3.49',
       '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49', '$10.00',
       '$24.99', '$11.99', '$79.99', '$16.99', '$14.99', '$29.99',
       '$12.99', '$2.49', '$10.99', '$1.50', '$19.99', '$15.99', '$33.99',
       '$39.99', '$3.95', '$4.49', '$1.70', '$8.99', '$1.49', '$3.88',
       '$399.99', '$17.99', '$400.00', '$3.02', '$1.76', '$4.84', '$4.77',
       '$1.61', '$2.50', '$1.59', '$6.49', '$1.29', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$8.49', '$1.75', '$14.00', '$2.00',
       '$3.08', '$2.59', '$19.40', '$3.90', '$4.59', '$15.46', '$3.04',
       '$13.99', '$4.29', '$3.28', '$4.60', '$1.00', '$2.95', '$2.90',
       '$1.97', '$2.56', '$1.20'], dtype=object)

Since all prices are ordinal data without exceptions, we can tranform this column by removing the dollar sign and converting to numeric. We can create a new column called `Price Numerical` and drop the original column.

We will achieve our goal in three steps. Follow the instructions of each step below.

#### First we remove the dollar sign. Do this in the next cell by applying the `str.replace` function to the column to replace `$` with an empty string (`''`).

In [56]:
# Your code here:
google_missing_removed['Price Numerical']=google_missing_removed['Price'].str.replace('$','')

In [58]:
google_missing_removed['Price Numerical'].unique()

array(['0', '4.99', '3.99', '6.99', '7.99', '5.99', '2.99', '3.49',
       '1.99', '9.99', '7.49', '0.99', '9.00', '5.49', '10.00', '24.99',
       '11.99', '79.99', '16.99', '14.99', '29.99', '12.99', '2.49',
       '10.99', '1.50', '19.99', '15.99', '33.99', '39.99', '3.95',
       '4.49', '1.70', '8.99', '1.49', '3.88', '399.99', '17.99',
       '400.00', '3.02', '1.76', '4.84', '4.77', '1.61', '2.50', '1.59',
       '6.49', '1.29', '299.99', '379.99', '37.99', '18.99', '389.99',
       '8.49', '1.75', '14.00', '2.00', '3.08', '2.59', '19.40', '3.90',
       '4.59', '15.46', '3.04', '13.99', '4.29', '3.28', '4.60', '1.00',
       '2.95', '2.90', '1.97', '2.56', '1.20'], dtype=object)

#### Second step, coerce the `Price Numerical` column to numeric.

In [60]:
# Your code here:
google_missing_removed['Price Numerical'] = pd.to_numeric(google_missing_removed['Price Numerical'])

**Finally, drop the original `Price` column.**

In [61]:
# Your code here:
google_missing_removed.drop(['Price'], axis=1, inplace=True)

Now check the variable types of `google_missing_removed`. Make sure:

* `Size` and `Price` columns have been removed.
* `Rating`, `Reviews`, and `Price Numerical` have the type of `float64`.
* `Last Updated` has the type of `datetime64`.

In [62]:
# Your code here
google_missing_removed.dtypes

App                        object
Category                   object
Rating                    float64
Reviews                   float64
Installs                   object
Type                       object
Content Rating             object
Genres                     object
Last Updated       datetime64[ns]
Current Ver                object
Android Ver                object
Reviews_numeric           float64
Reviews_isnull               bool
Price Numerical           float64
dtype: object

# Challenge 2 - Loading and Extracting Features from the Second Dataset

Load the second dataset to the variable `google_reviews`. The data is in the file `googleplaystore_user_reviews.csv`.

In [63]:
# Your code here:

google_review = pd.read_csv('../googleplaystore_user_reviews.csv')

#### This dataset contains the top 100 reviews for each app. 

Let's examine this dataset using the `head` function

In [64]:
# Your code here
google_review.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [66]:
google_review.shape

(64295, 5)

#### The main piece of information we would like to extract from this dataset is the proportion of positive reviews of each app. 

Columns like `Sentiment_Polarity` and `Sentiment_Subjectivity` are not to our interests because we have no clue how to use them. We do not care about `Translated_Review` because natural language processing is too complex for us at present (in fact the `Sentiment`, `Sentiment_Polarity`, and `Sentiment_Subjectivity` columns are derived from `Translated_Review` the data scientists). 

What we care about in this challenge is `Sentiment`. To be more precise, we care about **what is the proportion of *Positive* sentiment of each app**. This will require us to aggregate the `Sentiment` data by `App` in order to calculate the proportions.

Now that you are clear about what we are trying to achieve, follow the steps below that will walk you through towards our goal.

#### Our first step will be to remove all rows with missing sentiment. 

In the next cell, drop all rows with missing data using the `dropna()` function and assign this new dataframe to `review_missing_removed`.

In [65]:
# Your code here:
review_missing_removed=google_review.dropna(subset=['Sentiment'],  axis=0)

In [67]:
review_missing_removed.shape

(37432, 5)

#### Now, use the `value_counts()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html)) to get a sense on how many apps are in this dataset and their review counts.

In [69]:
# Your code here:
review_missing_removed['App'].value_counts()

Bowmasters                                            312
Angry Birds Classic                                   273
Helix Jump                                            273
Calorie Counter - MyFitnessPal                        255
Candy Crush Saga                                      240
Duolingo: Learn Languages Free                        240
Garena Free Fire                                      222
8 Ball Pool                                           219
Calorie Counter - Macros                              200
10 Best Foods for You                                 194
CBS Sports App - Scores, News, Stats & Watch Live     192
Google Photos                                         191
Alto's Adventure                                      175
8fit Workouts & Meal Planner                          171
DRAGON BALL LEGENDS                                   167
Candy Crush Soda Saga                                 166
Clash Royale                                          165
Adobe Acrobat 

#### Now the tough part comes. Let's plan how we will achieve our goal:

1. We will count the number of reviews that contain *Positive* in the `Sentiment` column.

1. We will create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app.

1. We will then loop the new dataframe to calculate the postivie review portion of each app.

#### Step 1: Count the number of positive reviews.

In the following cell, write a function that takes a column and returns the number of times *Positive* appears in the column. 

*Hint: One option is to use the `np.where()` function ([documentation](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html)).*

In [87]:
# Your code below

def positive_function(x):
    """
    Count how many times the string `Positive` appears in a column (exact string match).
    
    Args:
        x: data column
    
    Returns:
        The number of occurrences of `Positive` in the column data.
    """
    bla = x[x.str.contains('Positive')].shape[0]
    return bla

In [130]:
# Your code below

def positive_function2(x):
    """
    Count how many times the string `Positive` appears in a column (exact string match).
    
    Args:
        x: data column
    
    Returns:
        The number of occurrences of `Positive` in the column data.
    """
    bla = x.value_counts()[0]
    return bla

In [128]:
positive_function(review_missing_removed['Sentiment'])
#review_missing_removed['Sentiment'].value_counts().Positive

23998

#### Step 2: Create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app

We will group `review_missing_removed` by the `App` column, then aggregate the grouped dataframe on the number of positive reviews and the total review counts of each app. The result will be assigned to a new variable `google_agg`. Here is the ([documentation on how to achieve it](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)). Take a moment or two to read the documentation and google examples because it is pretty complex.

When you obtain `google_agg`, check its values to make sure it has an `App` column as its index as well as a `Positive` column and a `Total` column. Your output should look like:

![Positive Reviews Agg](positive-review-agg.png)

*Hint: Use `positive_function` you created earlier as part of the param passed to the `agg()` function in order to aggregate the number of positive reviews.*

#### Bonus:

As of Pandas v0.23.4, you may opt to supply an array or an object to `agg()`. If you use the array param, you'll need to rename the columns so that their names are `Positive` and `Total`. Using the object param will allow you to create the aggregated columns with the desirable names without renaming them. However, you will probably encounter a warning indicating supplying an object to `agg()` will become outdated. It's up to you which way you will use. Try both ways out. Any way is fine as long as it works.

In [147]:
# Your code here:
google_agg = review_missing_removed.groupby(['App'])['Sentiment'].agg([positive_function,np.size]).reset_index()

In [148]:
google_agg.head()

,App,positive_function,size
0,10 Best Foods for You,162,194
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,31,40
2,11st,23,40
3,1800 Contacts - Lens Store,64,80
4,1LINE – One Line with One Touch,27,38


In [132]:
google_agg

,App,positive_function2,size
0,10 Best Foods for You,162,194
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,31,40
2,11st,23,40
3,1800 Contacts - Lens Store,64,80
4,1LINE – One Line with One Touch,27,38
5,2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,25,32
6,21-Day Meditation Experience,68,80
7,"2Date Dating App, Love and matching",26,38
8,2GIS: directory & navigator,23,40
9,2RedBeans,31,39


Print the first 5 rows of `google_agg` to check it.

In [111]:
# Your code here
google_agg.head()

,App,positive_function,size
0,10 Best Foods for You,162,194
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,31,40
2,11st,23,40
3,1800 Contacts - Lens Store,64,80
4,1LINE – One Line with One Touch,27,38


#### Add a derived column to `google_agg` that is the ratio of the `Positive` and the `Total` columns. Call this column `Positive Ratio`. 

Make sure to account for the case where the denominator is zero using the `np.where()` function.

In [112]:
# Your code here:
google_agg['Positive Ration']=google_agg['positive_function']/google_agg['size']

In [113]:
google_agg.head()

,App,positive_function,size,Positive Ration
0,10 Best Foods for You,162,194,0.835052
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,31,40,0.775000
2,11st,23,40,0.575000
3,1800 Contacts - Lens Store,64,80,0.800000
4,1LINE – One Line with One Touch,27,38,0.710526


#### Now drop the `Positive` and `Total` columns. Do this with `inplace=True`.

In [114]:
# Your code here:
google_agg.drop(['positive_function','size'], axis=1, inplace=True)

Print the first 5 rows of `google_agg`. Your output should look like:

![Positive Reviews Agg](positive-review-ratio.png)

In [115]:
# Your code here:
google_agg.head()

,App,Positive Ration
0,10 Best Foods for You,0.835052
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.775000
2,11st,0.575000
3,1800 Contacts - Lens Store,0.800000
4,1LINE – One Line with One Touch,0.710526


# Challenge 3 - Join the Dataframes

In this part of the lab, we will join the two dataframes and obtain a dataframe that contains features we can use in our ML algorithm.

In the next cell, join the `google_missing_removed` dataframe with the `google_agg` dataframe on the `App` column. Assign this dataframe to the variable `google`.

In [116]:
google_missing_removed.dtypes

App                        object
Category                   object
Rating                    float64
Reviews                   float64
Installs                   object
Type                       object
Content Rating             object
Genres                     object
Last Updated       datetime64[ns]
Current Ver                object
Android Ver                object
Reviews_numeric           float64
Reviews_isnull               bool
Price Numerical           float64
dtype: object

In [143]:
google_agg.head()

,App,positive_function2,size
0,10 Best Foods for You,162,194
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,31,40
2,11st,23,40
3,1800 Contacts - Lens Store,64,80
4,1LINE – One Line with One Touch,27,38


In [121]:
# Your code here:
google = google_missing_removed.join(google_agg.set_index('App'), on='App', how='inner')

#### Let's look at the final result using the `head()` function. Your final product should look like:

![Final Product](google-final-head.png)

In [145]:
# Your code here:
google.sort_values(by='Positive Ration', ascending=False)

,App,Category,Rating,Reviews,Installs,Type,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Reviews_numeric,Reviews_isnull,Price Numerical,Positive Ration
440,"CallApp: Caller ID, Blocker & Phone Call Recorder",COMMUNICATION,4.4,483565.0,"10,000,000+",Free,Everyone,Communication,2018-07-29,1.286,4.1 and up,483565.0,False,0.00,1.000000
886,Amazon Prime Video,ENTERTAINMENT,4.2,411683.0,"50,000,000+",Free,Teen,Entertainment,2018-08-02,3.0.231.18141,4.1 and up,411683.0,False,0.00,1.000000
3620,Daniel Tiger for Parents,PARENTING,4.7,247.0,"100,000+",Free,Everyone,Parenting;Music & Video,2018-03-19,1.3.0,4.4 and up,247.0,False,0.00,1.000000
2169,All-in-One Mahjong 3 FREE,FAMILY,4.5,566.0,"50,000+",Free,Everyone,Board;Brain Games,2018-06-13,20180609,4.0 and up,566.0,False,0.00,1.000000
450,Caller ID +,COMMUNICATION,4.0,9498.0,"1,000,000+",Free,Everyone,Communication,2016-06-07,5.28.0,2.3 and up,9498.0,False,0.00,1.000000
4144,Google+,SOCIAL,4.2,4828372.0,"1,000,000,000+",Free,Teen,Social,2018-07-26,Varies with device,Varies with device,4828372.0,False,0.00,1.000000
812,HomeWork,EDUCATION,4.3,16195.0,"1,000,000+",Free,Everyone,Education,2016-09-20,8.5.2,4.0 and up,16195.0,False,0.00,1.000000
8305,"CallApp: Caller ID, Blocker & Phone Call Recorder",COMMUNICATION,4.4,483782.0,"10,000,000+",Free,Everyone,Communication,2018-07-29,1.286,4.1 and up,483782.0,False,0.00,1.000000
3848,GPS Speedometer and Odometer,MAPS_AND_NAVIGATION,4.8,15865.0,"1,000,000+",Free,Everyone,Maps & Navigation,2018-08-03,10.0,4.1 and up,15865.0,False,0.00,1.000000
2554,Google+,SOCIAL,4.2,4831125.0,"1,000,000,000+",Free,Teen,Social,2018-07-26,Varies with device,Varies with device,4831125.0,False,0.00,1.000000


<div class="alert alert-block alert-success">
<b>Tip:</b> Use green boxes (alert-success) for tips and notes. 
</div>